In [1]:
import pandas as pd

df = pd.read_csv("tubedata.csv", header=None) 
df.head()
    

,0,1,2,3,4,5
0,Harrow & Wealdstone,Kenton,Bakerloo,3,5,0
1,Kenton,South Kenton,Bakerloo,2,4,0
2,South Kenton,North Wembley,Bakerloo,2,4,0
3,North Wembley,Wembley Central,Bakerloo,2,4,0
4,Wembley Central,Stonebridge Park,Bakerloo,3,4,0


In [2]:
from collections import defaultdict

def creat_tube_map(df):
    station_dict = defaultdict(list) 
    zone_dict = defaultdict(set)
    station_line_dict = defaultdict(set)
    # get data row by row
    for index, row in df.iterrows():
        start_station = row[0] 
        end_station = row[1] 
        line = row[2]
        act_cost = int(row[3]) 
        zone1 = row[4]
        zone2 = row[5]
        # station dict. of child station tuples (child name, cost from parent to the child) 
        # {"Mile End": [("Stepney Green", 2), ("Wembley", 1)]}
        
        if (end_station, act_cost,line) not in station_dict[start_station]:
            station_dict[start_station].append((end_station, act_cost, line))
        station_line_dict[start_station].add(line)
        if (start_station, act_cost, line) not in station_dict[end_station]:
            station_dict[end_station].append((start_station, act_cost, line))
        station_line_dict[end_station].add(line)
        
        # we add the main zone
        zone_dict[start_station].add(zone1) # we add the secondary zone
        if zone2 != "0":
            zone_dict[start_station].add(zone2)
            # if the secondary zone is not 0 it’s the main zone for the ending station 
            zone_dict[end_station].add(zone2)
        else:
            # otherwise the main zone for the ending station is the same as the starting station 
            zone_dict[end_station].add(zone1)
            
    return station_dict, zone_dict, station_line_dict

In [3]:
# for station in station_dict.keys():
#         if station == "Lensington":
#             del station_dict['Lensington']
#         if station == "Wimbeldon":
#             del station_dict["Wimbeldon"]
#         if station == 'Kensington':
#             del station_dict["Kensington"]
#         if station == 'Elmers End':
#             del station_dict['Elmers End']
#         if station == 'Mitcham Junction':
#             del station_dict['Mitcham Junction']

In [4]:
station_dict, zone_dict, station_line_dict = creat_tube_map(df)
print(station_dict['Liverpool Street'])
# print('------------------------------------')
# print('------------------------------------')
# print(zone_dict)
# print('------------------------------------')
# print('------------------------------------')
# print(station_line_dict)

[('Bank/Monument', 2, 'Central'), ('Bethnal Green', 3, 'Central'), ('Moorgate', 2, 'Circle'), ('Aldgate', 2, 'Circle'), ('Moorgate', 2, 'Hammersmith & City'), ('Aldgate East', 4, 'Hammersmith & City'), ('Aldgate', 2, 'Metropolitan'), ('Moorgate', 2, 'Metropolitan')]


In [5]:
station_dict["Earls' Court"]

[('West Brompton', 2, 'District'),
 ('Olympia', 3, 'District'),
 ('High Street Kensington', 3, 'District'),
 ('West Kensington', 2, 'District'),
 ('Gloucester Road', 3, 'District'),
 ('Gloucester Road', 2, 'Piccadilly'),
 ('Barons Court', 3, 'Piccadilly')]

# 2-1 & 2-2 Implement and Compare DFS, BFS and USC

## 2-1-1 DFS

In [6]:
def dfs(graph, start, destination, already_visited, path, line, cost):
    path += [(start, line, cost)]
    ## Path structure: [(station, line, cost), (station, line, cost), ...]
    already_visited += [(start, line)]
    if start == destination:
        return path
    for neighbour, weight, line in graph[start]:
        if (neighbour, line) in already_visited:
            continue
        result = dfs(graph, neighbour, destination, already_visited, path, line=line, cost=weight)
        if result != None:
            return result
        
    ## current node reaches a dead end, so we remove it from path, and return None
    path.pop()
    return
            

In [7]:
## Test dfs
start_station = 'Canada Water'
end_station = 'Stratford'
already_visited = []
path = []
line = None
cost=0
final_path = dfs(station_dict, start_station, end_station, already_visited, path, line, cost)
print('This is the path from {} to {} founded by DFS:'.format(start_station, end_station))
print(final_path)
print('--------------------------------------')
print('Number of expanded nodes is:')
print(len(already_visited))
print('--------------------------------------')
print('The path cost is:')
print(sum([x[2] for x in final_path]))

This is the path from Canada Water to Stratford founded by DFS:
[('Canada Water', None, 0), ('Rotherhithe', 'East London', 1), ('Wapping', 'East London', 1), ('Shadwell', 'East London', 1), ('Whitechapel', 'East London', 2), ('Aldgate East', 'District', 2), ('Tower Hill', 'District', 2), ('Aldgate', 'Circle', 4), ('Liverpool Street', 'Circle', 2), ('Bank/Monument', 'Central', 2), ("St. Paul's", 'Central', 2), ('Chancery Lane', 'Central', 2), ('Holborn', 'Central', 1), ('Tottenham Court Road', 'Central', 2), ('Oxford Circus', 'Central', 2), ("Regent's Park", 'Bakerloo', 2), ('Baker Street', 'Bakerloo', 2), ('Marylebone', 'Bakerloo', 1), ('Edgware Road', 'Bakerloo', 2), ('Paddington', 'Bakerloo', 3), ('Edgware Road', 'Circle', 3), ('Paddington', 'Circle', 3), ('Bayswater', 'Circle', 2), ('Notting Hill Gate', 'Circle', 2), ('Holland Park', 'Central', 2), ("Shepherd's Bush", 'Central', 2), ('White City', 'Central', 3), ('East Acton', 'Central', 3), ('North Acton', 'Central', 2), ('West Act

In [8]:
import copy

## Now we calculate the search in reverse order
reverse_station_dict = copy.deepcopy(station_dict)
for station in reverse_station_dict.keys():
    reverse_station_dict[station].reverse()
    
start_station = 'Canada Water'
end_station = 'Stratford'
already_visited = []
path = []
line = None
cost=0
final_path = dfs(reverse_station_dict, start_station, end_station, already_visited, path, line, cost)
print('This is the path from {} to {} founded by DFS with reverse order:'.format(start_station, end_station))
print(final_path)
print('--------------------------------------')
print('Number of expanded nodes is:')
print(len(already_visited))
print('--------------------------------------')
print('The path cost is:')
print(sum([x[2] for x in final_path]))

This is the path from Canada Water to Stratford founded by DFS with reverse order:
[('Canada Water', None, 0), ('Canary Wharf', 'Jubilee', 3), ('North Greenwich', 'Jubilee', 3), ('Canning Town', 'Jubilee', 3), ('West Ham', 'Jubilee', 3), ('Stratford', 'Jubilee', 3)]
--------------------------------------
Number of expanded nodes is:
6
--------------------------------------
The path cost is:
15


## 2-1-2 BFS

In [9]:
def bfs(graph, start, destination, already_visited, line, cost):
    queue = []
    queue.append([(start, line, cost)])
    ## Queue structure: [[(station, line, cost), (station, line, cost), ()], [(),(),(),...]]
    while queue:
        ## Take first element
        path = queue.pop(0)
        ## Take the last node of the path to continue
        node = path[-1][0]
        line = path[-1][1]
        if (node, line) not in already_visited:
            already_visited.append((node, line))
            for neighbour, cost, line in graph[node]:
                # already_visited.append(neighbour)
                queue.append( path+[(neighbour, line, cost)] )
                if neighbour == destination:
                    already_visited.append((neighbour, line))
                    return path+[(neighbour, line, cost)]
    return None



In [10]:
## Test BFS
start_station = 'Canada Water'
end_station = 'Stratford'
already_visited = []
line = None
cost = 0
final_path = bfs(station_dict, start_station, end_station, already_visited, line, cost)
print('This is the path from {} to {} founded by BFS:'.format(start_station, end_station))
print(final_path)
print('--------------------------------------')
print('Number of expanded nodes is:')
print(len(already_visited))
print('--------------------------------------')
print('The path cost is:')
print(sum([x[2] for x in final_path]))

This is the path from Canada Water to Stratford founded by BFS:
[('Canada Water', None, 0), ('Canary Wharf', 'Jubilee', 3), ('North Greenwich', 'Jubilee', 3), ('Canning Town', 'Jubilee', 3), ('West Ham', 'Jubilee', 3), ('Stratford', 'Jubilee', 3)]
--------------------------------------
Number of expanded nodes is:
31
--------------------------------------
The path cost is:
15


In [11]:
## Test BFS Reverse
start_station = 'Canada Water'
end_station = 'Stratford'
already_visited = []
line = None
cost = 0
final_path = bfs(reverse_station_dict, start_station, end_station, already_visited, line, cost)
print('This is the path from {} to {} founded by BFS with reverse order:'.format(start_station, end_station))
print(final_path)
print('--------------------------------------')
print('Number of expanded nodes is:')
print(len(already_visited))
print('--------------------------------------')
print('The path cost is:')
print(sum([x[2] for x in final_path]))

This is the path from Canada Water to Stratford founded by BFS with reverse order:
[('Canada Water', None, 0), ('Canary Wharf', 'Jubilee', 3), ('North Greenwich', 'Jubilee', 3), ('Canning Town', 'Jubilee', 3), ('West Ham', 'Jubilee', 3), ('Stratford', 'Jubilee', 3)]
--------------------------------------
Number of expanded nodes is:
19
--------------------------------------
The path cost is:
15


## 2-1-3 UCS

In [12]:
from queue import PriorityQueue

def ucs(graph, start, destination, already_visited):
    p_queue = PriorityQueue()
    p_queue.put((0, [(start, None, 0)]))
    ## The structure of our Priority Queue is: [(path_cost1, [(station1, line, cost), (station2, line, cost), ...]), (path_cost2, [path2]), ...]
    
    while not p_queue.empty():
        ## Pick the element with lowest cost
        path_cost, path = p_queue.get()
        node = path[-1][0]
        line = path[-1][1]
        if node == destination:
            already_visited.append((node, line))
            return path, path_cost
        if (node, line) not in already_visited:
            already_visited.append((node, line))
            for neighbour, cost, line in graph[node]:
                new_path = path + [(neighbour, line, cost)]
                new_cost = path_cost + cost
                p_queue.put((new_cost, new_path))
                # if neighbour == destination:
                #     already_visited.append(neighbour)
                #     return new_path, new_cost
    return None

In [13]:
# from queue import PriorityQueue

# a = PriorityQueue()
# a.put((10,['a', 'b', 'c', 'd']))
# a.put((12,['b', 'g']))
# b, d = a.get()
# print(b)
# print(d)

In [14]:
## Test UCS
start_station = 'Canada Water'
end_station = 'Stratford'
already_visited = []
final_path, cost = ucs(station_dict, start_station, end_station, already_visited)
print('This is the path from {} to {} founded by UCS:'.format(start_station, end_station))
print(final_path)
print('--------------------------------------')
print('Number of expanded nodes is:')
print(len(already_visited))
print('--------------------------------------')
print('The path cost is:')
print(cost)

This is the path from Canada Water to Stratford founded by UCS:
[('Canada Water', None, 0), ('Rotherhithe', 'East London', 1), ('Wapping', 'East London', 1), ('Shadwell', 'East London', 1), ('Whitechapel', 'East London', 2), ('Stepney Green', 'District', 3), ('Mile End', 'District', 2), ('Stratford', 'Central', 4)]
--------------------------------------
Number of expanded nodes is:
106
--------------------------------------
The path cost is:
14


In [15]:
## UCS Reverse
start_station = 'Canada Water'
end_station = 'Stratford'
already_visited = []
final_path, cost = ucs(reverse_station_dict, start_station, end_station, already_visited)
print('This is the path from {} to {} founded by UCS with reverse order:'.format(start_station, end_station))
print(final_path)
print('--------------------------------------')
print('Number of expanded nodes is:')
print(len(already_visited))
print('--------------------------------------')
print('The path cost is:')
print(cost)

This is the path from Canada Water to Stratford founded by UCS with reverse order:
[('Canada Water', None, 0), ('Rotherhithe', 'East London', 1), ('Wapping', 'East London', 1), ('Shadwell', 'East London', 1), ('Whitechapel', 'East London', 2), ('Stepney Green', 'District', 3), ('Mile End', 'District', 2), ('Stratford', 'Central', 4)]
--------------------------------------
Number of expanded nodes is:
106
--------------------------------------
The path cost is:
14


# 2-3 Extending the cost function

In [16]:
def compute_path_cost(path, time_cost):
    curr_line = path[1][1]
    path_cost = 0
    for item in path[1:]:
        path_cost += item[2]
        if curr_line != item[1]:
            path_cost += time_cost
            curr_line = item[1]
    return path_cost

In [17]:
## Test dfs
start_station = 'Ealing Broadway'
end_station = 'South Kensington'
already_visited = []
path = []
line = None
cost = 0
final_path = dfs(station_dict, start_station, end_station, already_visited, path, line, cost)
print('This is the path from {} to {} founded by DFS:'.format(start_station, end_station))
print(final_path)
print('--------------------------------------')
print('Number of expanded nodes is:')
print(len(already_visited))
print('--------------------------------------')
print('The path cost is:')
print(compute_path_cost(final_path, time_cost=2))

This is the path from Ealing Broadway to South Kensington founded by DFS:
[('Ealing Broadway', None, 0), ('West Acton', 'Central', 3), ('Ealing Broadway', 'Central', 3), ('Ealing Common', 'District', 4), ('Acton Town', 'District', 2), ('Chiswick Park', 'District', 2), ('Turnham Green', 'District', 2), ('Stamford Brook', 'District', 1), ('Ravenscourt Park', 'District', 2), ('Hammersmith', 'District', 2), ('Barons Court', 'District', 1), ('West Kensington', 'District', 2), ("Earls' Court", 'District', 2), ('High Street Kensington', 'District', 3), ('Gloucester Road', 'Circle', 4), ('South Kensington', 'Circle', 1)]
--------------------------------------
Number of expanded nodes is:
29
--------------------------------------
The path cost is:
38


In [18]:
## Test bfs
start_station = 'Ealing Broadway'
end_station = 'South Kensington'
already_visited = []
line = None
cost = 0
final_path = bfs(station_dict, start_station, end_station, already_visited, line, cost)
print('This is the path from {} to {} founded by BFS:'.format(start_station, end_station))
print(final_path)
print('--------------------------------------')
print('Number of expanded nodes is:')
print(len(already_visited))
print('--------------------------------------')
print('The path cost is:')
print(compute_path_cost(final_path, time_cost=2))

This is the path from Ealing Broadway to South Kensington founded by BFS:
[('Ealing Broadway', None, 0), ('Ealing Common', 'District', 4), ('Acton Town', 'District', 2), ('Turnham Green', 'Piccadilly', 3), ('Hammersmith', 'Piccadilly', 3), ('Barons Court', 'District', 1), ("Earls' Court", 'Piccadilly', 3), ('Gloucester Road', 'District', 3), ('South Kensington', 'Circle', 1)]
--------------------------------------
Number of expanded nodes is:
53
--------------------------------------
The path cost is:
30


In [19]:
def ucs_time(graph, start, destination, already_visited, station_line_dict, time_cost):
    p_queue = PriorityQueue()
    p_queue.put((0, [(start, None, 0)]))
    ## The structure of our Priority Queue is: [(path_cost1, [(station1, line, cost), (station2, line, cost), ...]), (path_cost2, [path2]), ...]
    
    while not p_queue.empty():
        ## Pick the element with lowest cost
        path_cost, path = p_queue.get()
        node = path[-1][0]
        line = path[-1][1]
        if node == destination:
            already_visited.append((node, line))
            return path, path_cost
        if (node, line) not in already_visited:
            already_visited.append((node, line))
            for neighbour, cost, line in graph[node]:
                # already_visited.append(neighbour)
                new_path = path + [(neighbour, line, cost)]
                new_cost = path_cost + cost
                if line != path[-1][1] and path[-1][1] != None:
                    new_cost += time_cost
                p_queue.put((new_cost, new_path))
    return None

In [20]:
# min([('a', 45), ('b', 23), ('c', -1)], key=lambda tup: tup[1])

In [21]:
## Test UCS
start_station = 'Ealing Broadway'
end_station = 'South Kensington'
already_visited = []
final_path, cost = ucs_time(station_dict, start_station, end_station, already_visited, station_line_dict, time_cost=2)
print('This is the path from {} to {} founded by UCS:'.format(start_station, end_station))
print(final_path)
print('--------------------------------------')
print('Number of expanded nodes is:')
print(len(already_visited))
print('--------------------------------------')
print('The path cost is:')
print(cost)

This is the path from Ealing Broadway to South Kensington founded by UCS:
[('Ealing Broadway', None, 0), ('Ealing Common', 'District', 4), ('Acton Town', 'District', 2), ('Turnham Green', 'Piccadilly', 3), ('Hammersmith', 'Piccadilly', 3), ('Barons Court', 'Piccadilly', 2), ("Earls' Court", 'Piccadilly', 3), ('Gloucester Road', 'Piccadilly', 2), ('South Kensington', 'Piccadilly', 1)]
--------------------------------------
Number of expanded nodes is:
60
--------------------------------------
The path cost is:
22


# Heuristic

In [22]:
df = pd.read_csv("stations-coordination.csv") 
df.head()

,FID,OBJECTID,NAME,EASTING,NORTHING,LINES,NETWORK,Zone,x,y
0,0,78,Temple,530959,180803,"District, Circle",London Underground,1,-0.112644,51.510474
1,1,79,Blackfriars,531694,180893,"District, Circle",London Underground,1,-0.102020,51.511114
2,2,80,Mansion House,532354,180932,"District, Circle",London Underground,1,-0.092495,51.511306
3,3,81,Cannon Street,532611,180900,"District, Circle",London Underground,1,-0.088801,51.510963
4,4,82,Monument,532912,180824,"District, Circle",London Underground,1,-0.084502,51.510209


In [23]:
df.loc[ (df['NAME'].str.contains('Liverpool Street'))]['NORTHING'].values

array([181567, 181646, 181567])

In [24]:
def build_station_coord(station_dict, df):
    station_coord_dict = defaultdict(tuple)
    ## Some exceptions!
    station_coord_dict['Bank/Monument'] = (532912, 180824)
    station_coord_dict["St. James' Park"] = (529647, 179498)
    station_coord_dict["Earls' Court"] = (525534, 178552)
    station_coord_dict['Heathrow Terminal 3'] = (507586, 174430)
    station_coord_dict['Edgware Road'] = (527220, 181697)
    station_coord_dict['Paddington'] = (526459, 181527)
    station_coord_dict['Hammersmith'] = (523325, 178698)
    station_coord_dict['Olympia'] = (524342, 179162)
    station_coord_dict['Shoreditch'] = (533555, 182248)
        
    for station in station_dict.keys():
        if station in ['Lensington', 'Wimbeldon', 'Kensington', 'Elmers End', 'Mitcham Junction']:
            continue
        if station in ['Bank/Monument', "St. James' Park", "Earls' Court", 'Heathrow Terminal 3', 'Edgware Road',
                      'Paddington', 'Hammersmith', 'Olympia', 'Shoreditch']:
            continue
            
        x = df.loc[ (df['NAME'] == station) ]['EASTING'].values[0]
        y = df.loc[ (df['NAME'] == station) ]['NORTHING'].values[0]
        station_coord_dict[station] = (x, y)
    
    return station_coord_dict


In [25]:
## Building the dictionary for station coordination
station_coord_dict = build_station_coord(station_dict, df)

In [26]:
# station_coord_dict

In [27]:
import math


def heuristic(station_coord_dict, station, destination):
    x, y = station_coord_dict[station]
    target_x, target_y = station_coord_dict[destination]
    dist = math.dist([x, y], [target_x, target_y])
    return dist

In [28]:
# for station in station_dict.keys():
#     for neighbour, cost, line in station_dict[station]:
#         print(station, neighbour, line, cost, heuristic(station_coord_dict, station, neighbour))

In [29]:

def best_first_search(graph, start, destination, already_visited, station_line_dict, station_coord_dict, time_cost):
    p_queue = PriorityQueue()
    p_queue.put((0, [(start, None, 0)]))
    ## The structure of our Priority Queue is: [(heur_cost, [(station1, line, cost), (station2, line, cost), ...]), (Heuristic, [path2]), ...]
    
    while not p_queue.empty():
        ## Pick the element with lowest cost
        a, path = p_queue.get()
        node = path[-1][0]
        line = path[-1][1]
        if (node, line) not in already_visited:
            already_visited.append((node, line))
            for neighbour, cost, line in graph[node]:
                new_path = path + [(neighbour, line, cost)]
                heur_cost = heuristic(station_coord_dict, neighbour, destination)
                # if line != path[-1][1] and path[-1][1] != None:
                #     new_heur_cost += time_cost
                p_queue.put((heur_cost, new_path))
                if neighbour == destination:
                    already_visited.append(neighbour)
                    return new_path
    return None

In [30]:
start_station = 'Epping'
end_station = 'Dollis Hill'
already_visited = []
final_path = best_first_search(station_dict, start_station, end_station, already_visited, station_line_dict,
                                     station_coord_dict, time_cost=2)
print('This is the path from {} to {} founded by Best First Search:'.format(start_station, end_station))
print(final_path)
print('--------------------------------------')
print('Number of expanded nodes is:')
print(len(already_visited))
print('--------------------------------------')
print('The path cost is:')
print(compute_path_cost(final_path, time_cost=2))

This is the path from Epping to Dollis Hill founded by Best First Search:
[('Epping', None, 0), ('Theydon Bois', 'Central', 2), ('Debden', 'Central', 3), ('Loughton', 'Central', 2), ('Buckhurst Hill', 'Central', 2), ('Woodford', 'Central', 3), ('South Woodford', 'Central', 2), ('Snaresbrook', 'Central', 2), ('Leytonstone', 'Central', 2), ('Leyton', 'Central', 3), ('Stratford', 'Central', 2), ('Mile End', 'Central', 4), ('Bethnal Green', 'Central', 2), ('Liverpool Street', 'Central', 3), ('Moorgate', 'Circle', 2), ('Barbican', 'Circle', 2), ('Farringdon', 'Circle', 1), ("King's Cross St. Pancras", 'Circle', 4), ('Euston Square', 'Circle', 2), ('Great Portland Street', 'Circle', 2), ('Baker Street', 'Circle', 3), ('Finchley Road', 'Metropolitan', 6), ('Wembley Park', 'Metropolitan', 7), ('Neasden', 'Jubilee', 4), ('Dollis Hill', 'Jubilee', 2)]
--------------------------------------
Number of expanded nodes is:
48
--------------------------------------
The path cost is:
73


In [31]:
## Test UCS
start_station = 'Epping'
end_station = 'Dollis Hill'
already_visited = []
final_path, cost = ucs_time(station_dict, start_station, end_station, already_visited, station_line_dict, time_cost=2)
print('This is the path from {} to {} founded by UCS:'.format(start_station, end_station))
print(final_path)
print('--------------------------------------')
print('Number of expanded nodes is:')
print(len(already_visited))
print('--------------------------------------')
print('The path cost is:')
print(cost)

This is the path from Epping to Dollis Hill founded by UCS:
[('Epping', None, 0), ('Theydon Bois', 'Central', 2), ('Debden', 'Central', 3), ('Loughton', 'Central', 2), ('Buckhurst Hill', 'Central', 2), ('Woodford', 'Central', 3), ('South Woodford', 'Central', 2), ('Snaresbrook', 'Central', 2), ('Leytonstone', 'Central', 2), ('Leyton', 'Central', 3), ('Stratford', 'Central', 2), ('Mile End', 'Central', 4), ('Bethnal Green', 'Central', 2), ('Liverpool Street', 'Central', 3), ('Bank/Monument', 'Central', 2), ("St. Paul's", 'Central', 2), ('Chancery Lane', 'Central', 2), ('Holborn', 'Central', 1), ('Tottenham Court Road', 'Central', 2), ('Oxford Circus', 'Central', 2), ('Bond Street', 'Central', 1), ('Baker Street', 'Jubilee', 2), ("St. John's Wood", 'Jubilee', 4), ('Swiss Cottage', 'Jubilee', 1), ('Finchley Road', 'Jubilee', 2), ('West Hampstead', 'Jubilee', 1), ('Kilburn', 'Jubilee', 2), ('Willesden Green', 'Jubilee', 2), ('Dollis Hill', 'Jubilee', 2)]
-----------------------------------